# Example of DOV search methods based on external WFS

## Use cases

- Filter data within a certain community using its geographic borders
- Filter data within a geographic boundary of a feature in layer "Bekken"
- Filter data within a geographic boundary of a feature in layer "Hydrogeologische homogene zones"

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from owslib.etree import etree
from owslib.fes import PropertyIsEqualTo
from owslib.wfs import WebFeatureService

from pydov.search.boring import BoringSearch
from pydov.search.grondwaterfilter import GrondwaterFilterSearch

from pydov.util.location import (
    GmlFilter,
    Within,
)

# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform

# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    inProj = Proj(init='epsg:31370')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj, outProj, x1, y1)
    return x2, y2

# Filter data within a certain community using its geographic borders

This example will use a third party WFS service to retrieve to geometry of (the boundary of) a community and subsequently use this boundary as a spatial filter to query boreholes from the DOV service.

We have to start by defining the third party WFS service that contains the featuretype (layer) that contains the geometry we're interested in:

In [2]:
gemeentegrenzen = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VRBG/wfs',
    version='1.1.0')

In this case we know in advance which featuretype from the WFS service we need (`VRBG:RefGem`) but are unsure which field (attribute) we can use to get the community we want.

We can get the schema (i.e. all the available fields) of a layer to find a field to query on:

In [3]:
gemeentegrenzen.get_schema('VRBG:Refgem')['properties']

{'DATPUBLBS': 'date',
 'NAAM': 'string',
 'NISCODE': 'string',
 'NUMAC': 'string',
 'OIDN': 'decimal',
 'TERRID': 'decimal',
 'UIDN': 'decimal'}

We can now build a search query to get the feature (including geometry) of the community with the `NAAM` of `Lievegem`:

In [4]:
naam_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Lievegem')

In [5]:
gemeente_poly = gemeentegrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

And use this GML feature inside a `GmlFilter` query to find all boreholes that are located `Within` the community borders:

In [6]:
bs = BoringSearch()
df = bs.search(
    location=GmlFilter(gemeente_poly, Within),
    return_fields=('pkey_boring', 'x', 'y'))

Finally we can show the result on the map:

In [7]:
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y']))
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['pkey_boring'][loc]).add_to(marker_cluster)
fmap

# Filter data within a geographic boundary of a feature in layer "Bekken"

This example will use a third party WFS service to retrieve to geometry of a waterbody and subsequently use this boundary as a spatial filter to query groundwater screens from the DOV service.

We have to start by defining the third party WFS service that contains the featuretype (layer) that contains the geometry we're interested in:

In [8]:
bekkens = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VHAZones/wfs',
    version='1.1.0'
)

We can list all the available featuretypes (layers) in the service if we are unsure which one to use:

In [9]:
list(bekkens.contents)

['VHAZones:Bekken', 'VHAZones:Deelbekken', 'VHAZones:Vhazone']

We can also get the schema (i.e. all the available fields) of a layer to find a field to query on:

In [10]:
bekkens.get_schema('VHAZones:Bekken')['properties']

{'BEKNAAM': 'string',
 'BEKNR': 'short',
 'OIDN': 'decimal',
 'STRMGEB': 'string',
 'UIDN': 'decimal'}

We can get all the distinct values of a specific field:

In [11]:
tree = etree.fromstring(bekkens.getfeature('VHAZones:Bekken', propertyname='BEKNAAM').read().encode("utf8"))
set((i.text for i in tree.findall('.//{%s}BEKNAAM' % tree.nsmap['VHAZones'])))

{'Bekken Brugse polders',
 'Bekken Gentse kanalen',
 'Beneden-Scheldebekken',
 'Boven-Scheldebekken',
 'Demerbekken',
 'Denderbekken',
 'Dijlebekken',
 'Ijzerbekken',
 'Leiebekken',
 'Maasbekken',
 'Netebekken'}

We can now build a search query to get the feature (including geometry) of the waterbody with the `BEKNAAM` of `Bekken Brugse polders`:

In [12]:
naam_filter = PropertyIsEqualTo(propertyname='BEKNAAM', literal='Bekken Brugse polders')

In [13]:
bekken_poly = bekkens.getfeature(
    typename='VHAZones:Bekken',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

And use this GML feature inside a `GmlFilter` query to find all groundwater screens that are located `Within` the waterbody:

In [14]:
filter_search = GrondwaterFilterSearch()
df = filter_search.search(
    max_features = 100,
    location=GmlFilter(bekken_poly, Within),
    return_fields=('pkey_filter', 'x', 'y')
)

Finally we can show the result on the map:

In [15]:
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y']))
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [16]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['pkey_filter'][loc]).add_to(marker_cluster)
fmap